In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:100% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:14pt;}
div.text_cell_render.rendered_html{font-size:14pt;}
div.text_cell_render ul li, code{font-size:22pt; line-height:14px;}
div.output {font-size:14pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:14pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:14pt;padding:5px;}
table.dataframe{font-size:14px;}
</style>
"""))

In [11]:
import warnings
import os
import logging
# 경고 제거
warnings.filterwarnings('ignore')

# transformers 로깅 레벨 조정
logging.getLogger("transformers").setLevel(logging.ERROR)

# Hugging Face symlink 경고 제거
os.environ['HF_HUB_DISABLE_SYMLINKS_WARNING'] = '1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# from transformers import pipeline, logging as hf_logging
# hf_logging.set_verbosity_error()

# <span style="color:red">ch1_허깅페이스</span>
- Inference API 이용 : 모델의 결과를 server에서
- pipline() 이용 : 모델을 다운로드 받아 모델의 결과를 local에서

    * raw text -> tokenizer -> model -> [0.11, 0.55, 0.xx,~]  logits값으로 prediction 결과 출력
    
```

허깅페이스 transformers에서 지원하는 task
"sentiment-analysis" : "text-clssification"의 별칭(감정분석 전용으로 사용)
"text-clssification" : 감정분석, 뉴스분료, 리뷰 분류 등 일반적인 문장 분류
"zero-shot-clssification" : 레이블 없이 학습, 주어진 후보군 중에서 분류
"token-clssification" : 개체명 인식(NER:Named Entity REcognition)등 단위 라벨링
"ner" : "token-clssification" 의 별칭
"fill-mask" : 빈칸 채우기
"text-generation" : 텍스트 생성(GPT류 모델에 사용)
"text2text-generation" : 번역, 요약 등 입력 -> 출력 변환
"translation" : 번역
"summarization" : 텍스트 요약
"question-answering" : 주어진 context를 보고 질문에 답하기.
"image-to-text" : 그림을 설명
"image-classification" : 이미지 분류

```

## 1. 텍스트 기반 감정분석(긍정/부정)

- C:\사용자\내컴퓨터명\.cache\huggingface\hub 모델 다운로드

In [3]:
from transformers import pipeline
classifier = pipeline(task="sentiment-analysis",
                     model="distilbert/distilbert-base-uncased-finetuned-sst-2-english")
# https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english
classifier("I've been waiting for a HuggingFace course my whole life.")

Device set to use cpu


[{'label': 'POSITIVE', 'score': 0.9598049521446228}]

In [4]:
from transformers import pipeline
classifier = pipeline(task="text-classification",
                     model="distilbert/distilbert-base-uncased-finetuned-sst-2-english")
# 감정분석시 내용이 많으면 list로
classifier([
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!"
])

Device set to use cpu


[{'label': 'POSITIVE', 'score': 0.9598049521446228},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

In [5]:
classifier(["이 영화 정말 최고였어요. 감동적이고 연기가 대단해",
            "This movie was the best. It's touching, and the acting is amazing"])

[{'label': 'POSITIVE', 'score': 0.857815682888031},
 {'label': 'POSITIVE', 'score': 0.9998821020126343}]

In [6]:
classifier("이 물건 정말 사고 싶어요")

[{'label': 'POSITIVE', 'score': 0.8577604293823242}]

In [7]:
classifier(["I like you", "I hat you", "나 너가 싫어", "힘들어요"])

[{'label': 'POSITIVE', 'score': 0.9998695850372314},
 {'label': 'POSITIVE', 'score': 0.999488353729248},
 {'label': 'NEGATIVE', 'score': 0.599323034286499},
 {'label': 'POSITIVE', 'score': 0.8669533729553223}]

In [17]:
from transformers import pipeline
classifier = pipeline(task="text-classification",
                      model="matthewburke/korean_sentiment")
texts = ['나는 너가 좋아', "당신이 싫어요", "힘들어요", "오늘 기분이 최고야"]
result = classifier(texts)

Device set to use cpu


In [18]:
result

[{'label': 'LABEL_1', 'score': 0.9557897448539734},
 {'label': 'LABEL_0', 'score': 0.9092598557472229},
 {'label': 'LABEL_0', 'score': 0.9140233397483826},
 {'label': 'LABEL_1', 'score': 0.9714491367340088}]

In [19]:
for text, result in zip(texts, classifier(texts)):
    label = "긍정" if result['label']=='LABEL_1' else "부정"
    print(f"{text} => {label} : {result['score']:.4f}")

나는 너가 좋아 => 긍정 : 0.9558
당신이 싫어요 => 부정 : 0.9093
힘들어요 => 부정 : 0.9140
오늘 기분이 최고야 => 긍정 : 0.9714


## 2. 제로샷분류(zero-shot분류)
- 기계학습 및 자연어처리에서 각 개별 작업에 대한 특정 교육없이 작업을 수행할 수 있는 모형(비지도학습)

In [22]:
classifier = pipeline("zero-shot-classification",
                     # model="facebook/bart-large-mnli"
                     )
classifier(
    "I have a problem with my iphone that needs to be resolved asap!",
    candidate_labels=["urgent", "not urgent", "phone", "tablet", "computer"]
)

No model was supplied, defaulted to facebook/bart-large-mnli and revision d7645e1 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


{'sequence': 'I have a problem with my iphone that needs to be resolved asap!',
 'labels': ['urgent', 'phone', 'computer', 'not urgent', 'tablet'],
 'scores': [0.5227580070495605,
  0.45814019441604614,
  0.0142647260800004,
  0.0026850001886487007,
  0.002152054337784648]}

In [23]:
sequence_to_classify = "One day I will see the world"
candidate_labels = ['travel', 'cooking', 'dancing']
classifier(sequence_to_classify, candidate_labels)

{'sequence': 'One day I will see the world',
 'labels': ['travel', 'dancing', 'cooking'],
 'scores': [0.9941016435623169, 0.0031261250842362642, 0.0027722232043743134]}

## 3. text 생성

In [29]:
from transformers import pipeline, set_seed
# set_seed(2)
generation = pipeline("text-generation", "gpt2") # 텍스트 생성 gpt3부터는 허깅페이스없음
generation(
    "in this course. We will teach you how to",
    pad_token_id=generation.tokenizer.eos_token_id
) # pad_token_id 경고를 없애려고 setting

Device set to use cpu


[{'generated_text': 'in this course. We will teach you how to be a good writer, a good historian, a good historian, a good journalist, a good historian, and you will learn how to be a good teacher.\n\nBy the end of this course, you will have read over 60 books, written more than 500 articles, and written more than 250 articles for the New York Times. You will also have written over 1,500 books, more than 300 articles, and written over 2,000 articles for The New York Times.\n\nYou will have listened to over 3 million people in more than 40 languages. You will have studied over 3 million books. You will have learned over 2,000 languages. You will have written over 5,000 books. You will have contributed over 600 articles, more than 800 articles, and contributed over 400 articles for The New York Times.\n\nYou will have written over 100 articles. You will have written over 600 articles. You will have contributed over 600 articles. You will have a strong interest in history, in the history 

In [5]:
result = generation(
    "in this course. We will teach you how to",
    pad_token_id=generation.tokenizer.eos_token_id
) 
print(result[0]['generated_text'])

in this course. We will teach you how to make my both in the show. She's against the both in the show. Achievement of these curse both in the football what the curse both in the curse both in the show. She's against the curse both in the curse both in the show. Achievement of the curse both in the curse both in the show.
She's against the curse both in the show in the show in the show in the show in the show in the show in the show in the show in the show in the show in the show in the show in the show in the show in the show in the show in the show in the show in the curse both in the show in the show in the s


In [31]:
# generation = pipeline("text-generation", "gpt2")
result = generation(
    "이 과정은 다음과 같은 방법을 알려드려요. ",
    pad_token_id = generation.tokenizer.eos_token_id
)
print(result[0]['generated_text'])

이 과정은 다음과 같은 방법을 알려드려요. 한 해리기국 않누 이스현들는아로 지버 기상스현들는 타를 가키는 기상난 해스현들는 무는 기상난 해스현들는 무는 해스현들는 아타마 해스현들는 않맘를 같은 나목환들는 아타마 해스현들는 기상난 해스�


In [4]:
from transformers import pipeline, set_seed
generation = pipeline("text-generation", "skt/kogpt2-base-v2")
result = generation(
    "이 과정은 다음과 같은 방법을 알려드려요. ",
    pad_token_id = generation.tokenizer.eos_token_id,
    max_new_tokens = 200, # 생성할 최대 길이 (생성할 토큰 수)
    num_return_sequences=1, # 생성할 문장 갯수
    do_sample=True,   # 다양한 샘플 사용
    top_k=50,         # top-k 샘플링(확률 높은 상위 50개 토큰만 사용)
    top_p=0.95,       # 확률이 높은 순서대로 95%가 될 때까지의 단어들로만 후보로 사용
    temperature=1.0,  # 창의성 조절(낮을 수록 보수적)
    no_repeat_ngram_size=2  # 반복 방지
)
print(result[0]['generated_text'])

Device set to use cpu


이 과정은 다음과 같은 방법을 알려드려요. __"
우리는 어느 한 쪽이 자신의 의견을 표명했다고 가정해서 그 결정이 잘못된 것으로 결론지어질 수 있다는 사실을 미리 알고 있어야 한다.
어떤 결정이 옳을지라도 그 결정을 실행하기가 더욱 힘들 것이라는 것을 의미한다.
이때는 반드시 자신이 결정을 내리기 전에 먼저 상대방의 의사를 들은 다음 그 이유를 설명해주는 것이 좋다.
이렇게 해야 자기 의견의 타당성과 실현 가능성에 대해 확신을 갖게 되는 것이다.
다른 사람이 내 의견에 찬성하는 것은 자신이 내린 결정의 타당성을 확신할 수 있기 때문이다.
그런데 이 방법이 사실 옳은 것이 아닌가?
왜냐하면 상대방의 의견을 받아들인 후에 자신의 생각을 다시 진지하게 들으면서 상대방의 주장을 다시 듣는다면 이러한 과정을 통해 자신의 주장이 타당함을 알 수 있다고 보기 때문이다.
또한 그 반대의 경우도 마찬가지이다
모든 결정을 내린 사람들은 결정을 내리는 데 있어서 자신의 의견은 옳다고 생각한다.
그들은 자신이 정한 의견을 반드시 받아들여야만 한다.
우리는 결정을 내려면 여러 가지 다른 방법을 생각할 수 있다.
그럼에도 불구하고 그들은 결정을 내리려고 노력하지는 않는다.
모든 결정은 반드시 그 의견과 다른 의견을 가진 사람만을


## 4. 마스크(빈칸) 채우기

In [13]:
unmasker = pipeline(task='fill-mask',
                   model='distilbert/distilroberta-base')  # 마스크 채우기
unmasker("I'm going to hospital and meet a <mask>", top_k=2)                  

[{'score': 0.19275707006454468,
  'token': 3299,
  'token_str': ' doctor',
  'sequence': "I'm going to hospital and meet a doctor"},
 {'score': 0.06794589757919312,
  'token': 27321,
  'token_str': ' psychiatrist',
  'sequence': "I'm going to hospital and meet a psychiatrist"}]

In [15]:
# unmasker("병원에 가서 <mask>를 만날거예요")

In [16]:
unmasker("Hello, I'm a <mask> model.")

[{'score': 0.0629730075597763,
  'token': 265,
  'token_str': ' business',
  'sequence': "Hello, I'm a business model."},
 {'score': 0.038101598620414734,
  'token': 18150,
  'token_str': ' freelance',
  'sequence': "Hello, I'm a freelance model."},
 {'score': 0.03764132782816887,
  'token': 774,
  'token_str': ' role',
  'sequence': "Hello, I'm a role model."},
 {'score': 0.037326786667108536,
  'token': 2734,
  'token_str': ' fashion',
  'sequence': "Hello, I'm a fashion model."},
 {'score': 0.026023676618933678,
  'token': 24526,
  'token_str': ' Playboy',
  'sequence': "Hello, I'm a Playboy model."}]

In [17]:
unmasker("안녕, 나는 <mask> 모델이에요.", top_k=3)

[{'score': 0.2771560847759247,
  'token': 6,
  'token_str': ',',
  'sequence': '안녕, 나는, 모델이에요.'},
 {'score': 0.10971927642822266,
  'token': 35,
  'token_str': ':',
  'sequence': '안녕, 나는: 모델이에요.'},
 {'score': 0.1012372151017189,
  'token': 126,
  'token_str': ' –',
  'sequence': '안녕, 나는 – 모델이에요.'}]

In [18]:
unmasker = pipeline(task='fill-mask',
                   model='google-bert/bert-base-uncased')  # 마스크 채우기
unmasker("Hello, I'm a [MASK] model", top_k=2)    

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

[{'score': 0.06705848127603531,
  'token': 4827,
  'token_str': 'fashion',
  'sequence': "hello, i ' m a fashion model"},
 {'score': 0.05897250398993492,
  'token': 2047,
  'token_str': 'new',
  'sequence': "hello, i ' m a new model"}]

### ※ Inference API 사용

In [20]:
from dotenv import load_dotenv
import os
load_dotenv()
# os.environ['HF_TOKEN']
# 허깅페이스 토큰을 read권한으로 생성하여 .env에 추가(.gitignore파일도 추가)

True